<a href="https://colab.research.google.com/github/OfficiallyNikita/Crab-Age-Prediction-ML-Project/blob/main/AI_ML_Assignmnet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# In this Project, the idea is to develop a Machine Learning Model to predict if the age of the crab.
   # Using the dataset attached dataset file with name train, We have to Identify the type of problem statement, use all possible algorithms.
   # Also there is a testData file which is unseen data, so used saved model file to do the prediction of the records in testData file and save the output in the result.csv file.


# 1st Importing all the necessary liberaries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib


In [3]:
# 2nd Loading the training Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
train_data.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [6]:
# 3rd Exploring the Data Set

In [18]:
import numpy as np
# basic info
train_data.info()
# problem statement - predicting the age of a crab

np.sort(train_data['Age'].unique()) # values from 1 to 29-- continous values. Hence,Regression

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              74001 non-null  int64  
 1   Sex             74001 non-null  int64  
 2   Length          74001 non-null  float64
 3   Diameter        74001 non-null  float64
 4   Height          74001 non-null  float64
 5   Weight          74001 non-null  float64
 6   Shucked Weight  74001 non-null  float64
 7   Viscera Weight  74001 non-null  float64
 8   Shell Weight    74001 non-null  float64
 9   Age             74001 non-null  int64  
dtypes: float64(7), int64(3)
memory usage: 5.6 MB


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29])

In [8]:
print(train_data.isnull().sum())

id                0
Sex               0
Length            0
Diameter          0
Height            0
Weight            0
Shucked Weight    0
Viscera Weight    0
Shell Weight      0
Age               0
dtype: int64


In [21]:
train_data.describe()




,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,74001.000000,74001.000000,74001.000000,74001.000000,74001.000000,74001.000000,74001.000000,74001.000000,74001.000000,74001.000000
mean,37000.000000,1.055067,1.317438,1.024480,0.348089,23.384339,10.103826,5.058193,6.723684,9.968419
std,21362.392972,0.820634,0.287754,0.237393,0.092033,12.647921,5.617903,2.792643,3.584342,3.175589
min,0.000000,0.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.042524,0.042524,1.000000
25%,18500.000000,0.000000,1.150000,0.887500,0.300000,13.437663,5.712424,2.863300,3.968930,8.000000
50%,37000.000000,1.000000,1.375000,1.075000,0.362500,23.799405,9.908150,4.989512,6.931453,10.000000
75%,55500.000000,2.000000,1.537500,1.200000,0.412500,32.162508,14.033003,6.988152,9.071840,11.000000
max,74000.000000,2.000000,2.012815,1.612500,2.825000,80.101512,42.184056,21.545620,28.491248,29.000000


In [22]:
# Outliers detection (of each column)
train_data.columns



Index(['id', 'Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'Age'],
      dtype='object')

In [23]:
# a function to calculate iqr for each col
def outlier_detection(train_data,col):
  q1 = train_data[col].quantile(0.25)
  q3 = train_data[col].quantile(0.75)
  iqr = q3-q1
  l_limit = q1 - 1.5 * iqr
  u_limit = q3 + 1.5 * iqr
  return l_limit, u_limit



In [24]:
# checking % of outliers in each col
num_col = ['Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight','Viscera Weight', 'Shell Weight']

for col in num_col:
  l_limit, u_limit= outlier_detection(train_data, col) # passing the dataframe and the column name
  outliers = train_data[ (train_data[col] < l_limit) | (train_data[col] > u_limit)]  # getting the outliers
  print(f"{col}: outliers detection {(len(outliers)/len(train_data)) * 100 : .2f}% of data" )



Length: outliers detection  1.71% of data
Diameter: outliers detection  1.73% of data
Height: outliers detection  1.69% of data
Weight: outliers detection  0.34% of data
Shucked Weight: outliers detection  0.42% of data
Viscera Weight: outliers detection  0.38% of data
Shell Weight: outliers detection  0.54% of data


In [25]:
train_data.shape

(74001, 10)

In [26]:
# handling outliers ( total % of outliers < 10 % of data)
for col in num_col:
  l_limit,u_limit = outlier_detection(train_data,col)
  train_data = train_data[ (train_data[col] >= l_limit) & (train_data[col] <= u_limit)]

In [27]:
train_data.shape

(71264, 10)

In [28]:
train_data.tail(5)

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
73996,73996,2,1.3500,1.0625,0.3375,23.700182,9.879801,4.379998,7.229122,11
73997,73997,2,1.2500,0.9250,0.3625,18.696495,8.915918,3.869707,6.236890,9
73998,73998,1,0.7500,0.5500,0.2000,3.231843,1.445825,0.751262,0.992232,6
73999,73999,2,1.4500,1.0250,0.3875,27.073773,10.631063,4.479221,8.788345,13
74000,74000,1,1.2875,0.9625,0.3750,21.403872,8.093782,4.890289,5.769123,11


In [29]:
# re-indexing since outlier removal has made DataFrame have non-sequential index values
train_data = train_data.reset_index(drop=True)

In [34]:
train_data.tail(5)

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
71259,2,1.3500,1.0625,0.3375,23.700182,9.879801,4.379998,7.229122,11
71260,2,1.2500,0.9250,0.3625,18.696495,8.915918,3.869707,6.236890,9
71261,1,0.7500,0.5500,0.2000,3.231843,1.445825,0.751262,0.992232,6
71262,2,1.4500,1.0250,0.3875,27.073773,10.631063,4.479221,8.788345,13
71263,1,1.2875,0.9625,0.3750,21.403872,8.093782,4.890289,5.769123,11


In [35]:
train_data.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,1,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,0,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,1,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [44]:
# Encode the 'Sex' column using LabelEncoder (M=0, F=1, I=2)
le= LabelEncoder()
train_data['Sex'] = le.fit_transform(train_data['Sex'])



In [45]:
train_data.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,1,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,0,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,1,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [50]:
# x, y seperation
x = train_data.iloc[:, 0:8]
y = train_data['Age']

x.shape, y.shape

((71264, 8), (71264,))

In [48]:
train_data.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,1,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,0,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,1,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [52]:
# Select features and target variable
X = train_data.drop(columns=['Age'])  # Features
y = train_data['Age']  # Target variable (Age)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [53]:
# Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [55]:
# Function to train and evaluate models
def evaluate_model(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    r2 = r2_score(y_val, predictions)
    mae = mean_absolute_error(y_val, predictions)
    mse = mean_squared_error(y_val, predictions)
    rmse = mse ** 0.5
    return {'R2 Score': r2, 'MAE': mae, 'MSE': mse, 'RMSE': rmse}


In [57]:
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
} #initializing models

In [58]:
# Evaluate each model and display the results
results = {name: evaluate_model(model, X_train, y_train, X_val, y_val) for name, model in models.items()}
results

{'Linear Regression': {'R2 Score': 0.5179871126914446,
  'MAE': 1.4890308205386737,
  'MSE': 4.420622739823906,
  'RMSE': 2.102527702510458},
 'Decision Tree': {'R2 Score': 0.03697082197262658,
  'MAE': 2.0126289202273204,
  'MSE': 8.832105521644566,
  'RMSE': 2.9718858527279552},
 'Random Forest': {'R2 Score': 0.5209203117318151,
  'MAE': 1.4599705325194696,
  'MSE': 4.393721869080194,
  'RMSE': 2.0961206714023395},
 'Gradient Boosting': {'R2 Score': 0.5518068784292602,
  'MAE': 1.4042642704852109,
  'MSE': 4.1104558762138845,
  'RMSE': 2.0274259237303554}}

In [59]:
 # Save the best-performing model
 best_model = models['Gradient Boosting']
joblib.dump(best_model, 'gradient_boosting_model.pkl')

['gradient_boosting_model.pkl']

In [60]:
# Load the test data (testData.xlsx)
test_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/testdata.xlsx')

In [63]:
# Encode the 'Sex' column using the same LabelEncoder
test_data['Sex'] = le.fit_transform(test_data['Sex'])

In [64]:
# Scale the features using the same StandardScaler
test_features = test_data.drop(columns=['id'])
test_features_scaled = scaler.transform(test_features)

In [65]:
# Load the saved Gradient Boosting model and make predictions
loaded_model = joblib.load('gradient_boosting_model.pkl')
predictions = loaded_model.predict(test_features_scaled)

# Save the predictions along with the 'id' column in a new CSV file
test_data['Age_Predicted'] = predictions
test_data[['id', 'Age_Predicted']].to_csv('result.csv', index=False)


In [66]:
from google.colab import files
files.download('result.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>